In [1]:
import panda_dataset as pds
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

from PIL import Image
from IPython import display

In [2]:
def as_gif(images, path="sample.gif"):
    images[0].save(path, save_all=True, append_images=images, duration=int(1000/30), loop=0)
    gif_bytes = open(path,"rb").read()
    return gif_bytes

In [4]:
panda_dataset = pds.PandaDataset(root_dir="./panda_dataset", transform=pds.ToTensor())
dataloader = DataLoader(panda_dataset, batch_size=1, num_workers=0)
images = []
for i_batch, sample_batched in enumerate(dataloader):
    timestamps = sample_batched["timestamps"]
    cam_in = sample_batched["calibration_info"]["intrinsic"][0]
    cam_ex = sample_batched["calibration_info"]["extrinsic"][0]
    # print(f"Camera Intrinsics:\n{cam_in}")
    # print(f"Camera Extrinsics:\n{cam_ex}")
    for i, timestamp in enumerate(sample_batched["steps"]):
        step = sample_batched["steps"][timestamp]
        image = step["img_data"].squeeze().permute(1,2,0).numpy()
        images.append(Image.fromarray(image))
    as_gif(images)
    break
